# Assignment #2: Language models
Author: Pierre Nugues

## Objectives

The objectives of this assignment are to:
* Write a program to find n-gram statistics
* Compute the probability of a sentence
* Know what a language model is
* Write a short report of 1 to 2 pages on the assignment

## Submission

Once you have written all the missing code and run all the cells, you will show it to an instructor that will pass you.

## Organization

* Each group will have to write Python programs to count unigrams, bigrams, and trigrams in a corpus of approximately one million words and to determine the probability of a sentence.
* You can test you regular expression using the regex101.com site
* Each student will have to write a short report of one to two pages and comment briefly the results. In your report, you must produce the tabulated results of your analysis as described below.

## Programming

### Imports

Some imports you may need. Add others as needed.

In [110]:
import bz2
import math
import os
import regex as re
import requests
import sys
from zipfile import ZipFile

### Collecting and analyzing a corpus

Retrieve a corpus of novels by Selma Lagerl&ouml;f from this URL:
<a href="https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt">
    <tt>https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt</tt>
</a>. The text of these novels was extracted
from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at
<a href="https://litteraturbanken.se/">Litteraturbanken</a>.

In [111]:
# You may have to adjust the path
corpus = open('Selma.txt', encoding='utf8').read()

Run the <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch02/python">concordance
program </a> to print the lines containing a specific word, for instance <i>Nils</i>.

In [112]:
pattern = 'Nils Holgersson'
width = 25

In [113]:
# spaces match tabs and newlines
pattern = re.sub(' ', r'\\s+', pattern)
# Replaces newlines with spaces in the text
clean_corpus = re.sub(r'\s+', ' ', corpus)
concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
               .format(pattern=pattern, width=width))
for match in re.finditer(concordance, clean_corpus):
    print(match.group(1))
# print the string with 0..width characters on either side

Selma Lagerlöf Nils Holgerssons underbara resa genom Sv
! Se på Tummetott! Se på Nils Holgersson Tummetott!» Genast vände
r,» sade han. »Jag heter Nils Holgersson och är son till en husma
lden. »Inte är det värt, Nils Holgersson, att du är ängslig eller
 i dem. På den tiden, då Nils Holgersson drog omkring med vildgäs
ulle allt visa honom vad Nils Holgersson från Västra Vemmenhög va
om ägde rum det året, då Nils Holgersson for omkring med vildgäss
m vad det kan kosta dem. Nils Holgersson hade inte haft förstånd 
de det inte mer sägas om Nils Holgersson, att han inte tyckte om 
 Rosenbom?» För där stod Nils Holgersson mitt uppe på Rosenboms n
 Med ens fingo de syn på Nils Holgersson, och då sköt den store v
vila. När vildgässen och Nils Holgersson äntligen hade letat sig 
 slags arbetare. Men vad Nils Holgersson inte såg, det var, att s
nde han fråga, och om då Nils Holgersson sade nej, började han ge
de lille Mats, och om nu Nils Holgersson också hade tegat, så had
åg så försmädlig ut,

Run a simple <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">tokenization
program</a> on your corpus.

In [114]:
def tokenize(text):
    words = re.findall(r'\p{L}+', text)
    return words

In [115]:
words = tokenize(corpus)
words[:10]

['Selma',
 'Lagerlöf',
 'Nils',
 'Holgerssons',
 'underbara',
 'resa',
 'genom',
 'Sverige',
 'Första',
 'bandet']

Count the number of unique words in the original corpus and when setting all the words in lowercase

Original text

In [116]:
# Write your code here
len(set(words))


44256

Lowercased text

In [117]:
# Write your code here
len(set(tokenize(corpus.lower())))

41032

### Segmenting a corpus

You will write a program to tokenize your text, insert `<s>` and `</s>` tags to delimit sentences, and set all the words in lowercase letters. In the end, you will only keep the words.

#### Normalizing 

Write a regular expression that matches all the characters that are neither a letter nor a punctuation sign. The punctuations signs will be the followings: `.;:?!`. In your regex, use the same order. For the definition of a letter, use a Unicode regex. You will call the regex string `nonletter`

In [118]:
# Write your code
nonLetter = r'[^\p{L}.;:?!]+'

Write a `clean()` function that replaces all the characters that are neither a letter nor a punctuation sign with a space. The punctuations signs will be the followings: `.;:?!`.   For the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

the result will be:

`En gång hade de på Mårbacka en barnpiga som hette Back Kajsa.`

In [119]:
# Write your code here
def clean(text: str):
    return re.sub(nonLetter, ' ', text)
 

In [120]:
test_para = 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa. \
Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, \
hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, \
när hon kammade dem, och till humöret var hon dyster och sorgbunden.'

In [121]:
test_para = clean(test_para)
test_para

'En gång hade de på Mårbacka en barnpiga som hette Back Kajsa. Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden.'

In [122]:
print(corpus[:200])

Selma Lagerlöf


Nils Holgerssons underbara resa genom Sverige. Första bandet





Bokutgåva


Albert Bonniers förlag, Stockholm 1907.




DEN KRISTLIGA DAGVISAN.





Den signade dag, som vi nu här s


In [123]:
print(clean(corpus[:200]))

Selma Lagerlöf Nils Holgerssons underbara resa genom Sverige. Första bandet Bokutgåva Albert Bonniers förlag Stockholm . DEN KRISTLIGA DAGVISAN. Den signade dag som vi nu här s


#### Segmenter

In this section, you will write a sentence segmenter that will delimit each sentence with `</s>` and `<s>` symbols. For example the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

will be bracketed as:

`<s> En gång hade de på Mårbacka en barnpiga som hette Back-Kajsa </s>`

As algorithm, you will use a simple heuristics to detect the sentence boundaries: A sentence starts with a capital letter and ends with a period-equivalent punctuation sign. You will write a regex to match these boundaries with a regular expression and you will insert `</s>\n<s>` symbols with a substitution function.

Write a regular expression that matches a punctuation, a sequence of spaces, and an uppercase letter. Call this regex string `sentence_boundaries`. In the regex, you will remember the value of the uppercase letter using a backreference. Use the Unicode regexes for the letters and the spaces. Do not use `\s`.

##### Detecting sentence boundaries

In [124]:
# Write your code here
sentence_boundaries = r'(\p{P})(\p{Z}+)(\p{Lu})'

##### Replacement markup

Write a string to replace the matched boundaries with the sentence boundary markup. Remember that a sentence ends with `</s>` and starts with `<s>` and that there is one sentence per line. Hint: The markup is `</s>\n<s>`. Remember also that the first letter of your sentence is in a regex backreference. Call the regex string `sentence_markup`.

In [125]:
# Write your code here
def sentences(text: str):
    return re.sub(sentence_boundaries, r" </s>\n<s> \3", clean(text))

##### Applying the substitution

Use your regexes to segment your text. Use the string `sentence_boundaries`, `sentence_markup`, and `test_para` as input and `text` as output.

In [126]:
# Write your code here
text = sentences(test_para)

In [127]:
print(text)

En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden.


Insert markup codes in the beginning and end of the text

In [128]:
# Write your code here
text1 = "<s>" + text + " <\s>"

In [129]:
print(text1)

<s>En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden. <\s>


Replace the space duplicates with one space and remove the punctuation signs. For the spaces, use the Unicode regex. Do not use `\s`.

In [130]:
# Write your code here
def reps(tex: str):
    return re.sub(r'[.,:;?!][" "]*', r" ", tex)

In [131]:
trgrg = reps(text1)
print(trgrg)

<s>En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden <\s>


Write a `segment_sentences(text)` function to gather the code in the Segmenter section and set the text in lowercase

In [132]:
# Write your code here
def segment_sentences(t):
    t = "<s> " +sentences(t) + " </s>"
    t =  reps(t)
    return t.lower()

In [133]:

print(segment_sentences(test_para))

<s> en gång hade de på mårbacka en barnpiga som hette back kajsa </s>
<s> hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


Estimate qualitatively the accuracy of your program.

#### Tokenizing the corpus

Clean and segment the corpus using the functions you have written

In [134]:
# Write your code here
corpus2 = segment_sentences(corpus)

The result should be a normalized text without punctuation signs where all the sentences are delimited with `<s>` and `</s>` tags. The five last lines of the text should look like this. You may have some small differences.

In [135]:
print(corpus2[-557:])

<s> hon hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse </s>
<s> då prästen sade detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg </s>
<s> prästens ord tycktes redan ha gått i uppfyllelse </s>
<s> där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad </s>
<s> hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare </s>


You will now create a list of words from your string. You will consider that a space or a carriage return is an item separator

In [136]:
# Write your code here
words = re.split('\s|\+', corpus2)

The five last lines of the corpus should like this:

In [137]:
#print(words)

### Counting unigrams and bigrams

Read and try programs to compute the frequency of unigrams and bigrams of the training set: [<a
            href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>].

#### Unigrams

In [138]:
def unigrams(words):
    frequency = {}
    for i in range(len(words)):
        if words[i] in frequency:
            frequency[words[i]] += 1
        else:
            frequency[words[i]] = 1
    return frequency

We compute the frequencies.

In [139]:
frequency = unigrams(words)
list(frequency.items())[:20]

[('<s>', 59047),
 ('selma', 52),
 ('lagerlöf', 270),
 ('nils', 87),
 ('holgerssons', 6),
 ('underbara', 23),
 ('resa', 317),
 ('genom', 688),
 ('sverige', 56),
 ('</s>', 59047),
 ('första', 525),
 ('bandet', 6),
 ('bokutgåva', 11),
 ('albert', 15),
 ('bonniers', 11),
 ('förlag', 11),
 ('stockholm', 77),
 ('', 336),
 ('den', 11624),
 ('kristliga', 2)]

#### Bigrams

In [140]:
def bigrams(words):
    bigrams = []
    for i in range(len(words) - 1):
        bigrams.append((words[i], words[i + 1]))
    frequency_bigrams = {}
    for i in range(len(words) - 1):
        if bigrams[i] in frequency_bigrams:
            frequency_bigrams[bigrams[i]] += 1
        else:
            frequency_bigrams[bigrams[i]] = 1
    return frequency_bigrams

In [141]:
frequency_bigrams = bigrams(words)
list(frequency_bigrams.items())[:20]

[(('<s>', 'selma'), 8),
 (('selma', 'lagerlöf'), 11),
 (('lagerlöf', 'nils'), 1),
 (('nils', 'holgerssons'), 6),
 (('holgerssons', 'underbara'), 4),
 (('underbara', 'resa'), 4),
 (('resa', 'genom'), 6),
 (('genom', 'sverige'), 5),
 (('sverige', '</s>'), 17),
 (('</s>', '<s>'), 59046),
 (('<s>', 'första'), 11),
 (('första', 'bandet'), 1),
 (('bandet', 'bokutgåva'), 2),
 (('bokutgåva', 'albert'), 11),
 (('albert', 'bonniers'), 11),
 (('bonniers', 'förlag'), 11),
 (('förlag', 'stockholm'), 10),
 (('stockholm', ''), 10),
 (('', '</s>'), 224),
 (('<s>', 'den'), 1375)]

In the report, tell what is the possible number of bigrams and their real number? Explain why such a difference. What would be the possible number of 4-grams.

Propose a solution to cope with bigrams unseen in the corpus.

### Computing the likelihood of a sentence
You will now compute the likelihood of a sentence using a bigram and a trigram models. 

In both models, you will ignore the start of sentence symbol, `<s>`, as this factor is common to both models: $P(<s>)$. This will save you one multiplication.

#### Unigrams

Write a program to compute a sentence's probability using unigrams. You may find useful the dictionaries that we saw in the mutual information program: [<a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>]. Your function will return the perplexity.

Your function should print and tabulate the results as in the examples below with the sentence _Det var en gång en katt som hette Nils_.

Your figures might be slightly different because of differences in the sentence segmentation.

```
=====================================================
wi 	 C(wi) 	 #words 	 P(wi)
=====================================================
det 	 21108 	 1041631 	 0.0202643738521607
var 	 12090 	 1041631 	 0.01160679741674355
en 	 13514 	 1041631 	 0.01297388422579589
gång 	 1332 	 1041631 	 0.001278763784871994
en 	 13514 	 1041631 	 0.01297388422579589
katt 	 16 	 1041631 	 1.5360525944408337e-05
som 	 16288 	 1041631 	 0.015637015411407686
hette 	 97 	 1041631 	 9.312318853797554e-05
nils 	 87 	 1041631 	 8.352285982272032e-05
</s> 	 59047 	 1041631 	 0.056687060964967444
=====================================================
Prob. unigrams:	 5.361459667285409e-27
Geometric mean prob.: 0.0023600885848765307
Entropy rate:	 8.726943273141258
Perplexity:	 423.71290908655254
```

In [142]:
# Write your code
def unigram_lm(frequency, sent_words):
    nbr_words = len(words)
    result = []
    result.append(["wi", "C(wi)", "#words", "P(wi)"])
    prob_unigrams = 1
    geometric_prob = 0
    for w in sent_words:
        freq = 0    
        if frequency.get(w) is not None:
            freq = frequency.get(w)
        p_wi = int(freq)/ nbr_words
        prob_unigrams = prob_unigrams * p_wi
        geometric_prob += p_wi
        result.append(
            [w, 
             freq, 
             nbr_words, 
             p_wi])
    
    geometric_prob = prob_unigrams ** (1.0/len(sent_words))  
    entropy_rate = - math.log2(prob_unigrams)/ len(sent_words)
    perplexity_unigrams = 2 ** (entropy_rate)

    matrix = result
    s = [[str(e) for e in row] for row in matrix]
    lens = [max(map(len, col)) for col in zip(*s)]
    fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
    table = [fmt.format(*row) for row in s]
    print('\n'.join(table))

    print("prob. unigrams: ",prob_unigrams)
    print("Geometrc mean prob.",geometric_prob)
    print("Entropy rate: ", entropy_rate)
    print("Perplexity: ",perplexity_unigrams)
    return perplexity_unigrams

In [143]:
sentence = 'det var en gång en katt som hette nils </s>'
sentence2 = '<s> det var en gång en katt som hette nils </s>'
sent_words = sentence.split()
sent_words2 = sentence2.split()
print(type((len(words))))
sent_words

<class 'int'>


['<s>',
 'det',
 'var',
 'en',
 'gång',
 'en',
 'katt',
 'som',
 'hette',
 'nils',
 '</s>']

In [144]:
perplexity_unigrams = unigram_lm(frequency, sent_words)

wi   	C(wi)	#words 	P(wi)                 
<s>  	59047	1041915	0.05667160948829799   
det  	21108	1041915	0.020258850290090842  
var  	12090	1041915	0.011603633693727416  
en   	13514	1041915	0.012970347869068014  
gång 	1332 	1041915	0.0012784152258101668 
en   	13514	1041915	0.012970347869068014  
katt 	16   	1041915	1.5356339048770772e-05
som  	16288	1041915	0.015632753151648648  
hette	97   	1041915	9.309780548317282e-05 
nils 	87   	1041915	8.350009357769108e-05 
</s> 	59047	1041915	0.05667160948829799   
prob. unigrams:  3.0301536479032274e-28
Geometrc mean prob. 0.003150032018730576
Entropy rate:  8.310417791595865
Perplexity:  317.45709061172874


In [145]:
perplexity_unigrams = int(perplexity_unigrams)
perplexity_unigrams

317

#### Bigrams

Write a program to compute the sentence probability using bigrams. Your function will tabulate and print the results as below. It will return the perplexity.

```
=====================================================
wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
=====================================================
<s>	 det 	 5672 	 59047 	 0.09605907158704083
det 	 var 	 3839 	 21108 	 0.1818741709304529
var 	 en 	 712 	 12090 	 0.058891645988420185
en 	 gång 	 706 	 13514 	 0.052242119283705785
gång 	 en 	 20 	 1332 	 0.015015015015015015
en 	 katt 	 6 	 13514 	 0.0004439840165754033
katt 	 som 	 2 	 16 	 0.125
som 	 hette 	 45 	 16288 	 0.002762770137524558
hette 	 nils 	 0 	 97 	 0.0 	 *backoff: 	 8.352285982272032e-05
nils 	 </s> 	 2 	 87 	 0.022988505747126436
=====================================================
Prob. bigrams:	 2.376007803503683e-19
Geometric mean prob.: 0.013727289294133601
Entropy rate:	 6.186809422848149
Perplexity:	 72.84759420254609
```

In [167]:
# Write your code
def bigram_lm(frequency, frequency_bigrams, sent_words):
    result = []
    result.append(["wi", "wi+1", "ci,i+1", "c(i)", "P(wi+1|wi)"])
    prob_bigrams = 1
    for i in range(len(sent_words) - 1):
        wi = sent_words[i]
        wi1 = sent_words[i + 1]
        t = (wi, wi1)
        freq = 0
        if frequency_bigrams.get(t) is not None:
            freq = frequency_bigrams.get(t)
        
        freq_wi = frequency.get(wi)
        pwi1wi = freq/freq_wi
        prob_bigrams = prob_bigrams *  (pwi1wi == 0 and 8.352285e-05 or pwi1wi)
        result.append([wi, wi1 ,freq, freq_wi , pwi1wi])

    geometric_prob = prob_bigrams ** (1.0/(len(sent_words)-1))  
    entropy_rate = -math.log2(prob_bigrams)/(len(sent_words)-1)
    perplexity_bigrams = 2 ** (entropy_rate)

    matrix = result
    s = [[str(e) for e in row] for row in matrix]
    lens = [max(map(len, col)) for col in zip(*s)]
    fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
    table = [fmt.format(*row) for row in s]
    print('\n'.join(table))

    print("prob. unigrams: ",prob_bigrams)
    print("Geometrc mean prob.",geometric_prob)
    print("Entropy rate: ", entropy_rate)
    print("Perplexity: ",perplexity_bigrams)
    return perplexity_bigrams

In [168]:
perplexity_bigrams = bigram_lm(frequency, frequency_bigrams, sent_words2)

wi   	wi+1 	ci,i+1	c(i) 	P(wi+1|wi)           
<s>  	det  	5672  	59047	0.09605907158704083  
det  	var  	3839  	21108	0.1818741709304529   
var  	en   	712   	12090	0.058891645988420185 
en   	gång 	706   	13514	0.052242119283705785 
gång 	en   	20    	1332 	0.015015015015015015 
en   	katt 	6     	13514	0.0004439840165754033
katt 	som  	2     	16   	0.125                
som  	hette	45    	16288	0.002762770137524558 
hette	nils 	0     	97   	0.0                  
nils 	</s> 	2     	87   	0.022988505747126436 
prob. unigrams:  2.376007524072876e-19
Geometrc mean prob. 0.013727289132693572
Entropy rate:  6.186809439814989
Perplexity:  72.8475950592715


In [148]:
perplexity_bigrams = int(perplexity_bigrams)
perplexity_bigrams

49

In addition to this sentence, _Det var en gång en katt som hette Nils_, write three other sentences that will form your test set and run your programs on them. You will insert them in your report.

### Online prediction of words

You will now carry out an online prediction of words. You will consider two cases:
1. Prediction of the current word a user is typing;
2. Prediction of the next word.

Ideally, you would write a loop that reads the words and apply the models while typing. As the Jupyter labs are not designed for interactive input and output, we will simplify the experimental settings with constant strings at a given time of the input.  

We will assume the user is typing the phrase: _Det var en gång_. 

#### Trigrams

To have a more accurate prediction, you will use a trigram counting function. Program this function following the model of your bigram counting function.

In [149]:
# Write your code
def trigrams(words):
    trigrams = []
    for i in range(len(words) - 2):
        trigrams.append((words[i], words[i + 1], words[i + 2]))
    frequency_trigrams = {}
    for i in range(len(words) - 2):
        if trigrams[i] in frequency_trigrams:
            frequency_trigrams[trigrams[i]] += 1
        else:
            frequency_trigrams[trigrams[i]] = 1
    return frequency_trigrams

In [150]:
frequency_trigrams = trigrams(words)
frequency_trigrams[('det', 'var', 'en')]

330

#### Prediction

The user starts typing _Det var en gång_. After the 2nd character, your program tries to help the user with suggested words.

In [151]:
starting_text = 'De'.lower()
starting_text

'de'

Write a program to rank the five first candidates at this point. Assign these predictions in a list that you will call `current_word_predictions_1`. Note that you are starting a sentence and you can then use the bigram frequencies. Write a sorting key that will enable you to have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [152]:
cand_nbr = 5

In [153]:
# Write your code here
current_word_predictions_1 = []
for w in frequency.keys():
    if w.startswith(starting_text):
        current_word_predictions_1.append((w, frequency[w]))
    
current_word_predictions_1.sort(key=lambda x: x[1], reverse=True)
current_word_predictions_1 = current_word_predictions_1[:cand_nbr]


In [154]:
print(current_word_predictions_1)

[('det', 21108), ('de', 11942), ('den', 11624), ('dem', 3196), ('detta', 2482)]


Let us now suppose that the user has typed: _Det var en_. After detecting a space, your program starts predicting a next possible word.

In [155]:
current_text = "Det var en ".lower()
current_text

'det var en '

Tokenize this text and return a list of tokens. Call it `tokens`.

In [156]:
# Write your code here
tokens = tokenize(current_text)

In [157]:
tokens

['det', 'var', 'en']

Write a program to propose the five next possible words ranked by frequency using a trigram model. Assign these predictions to a variable that you will call `next_word_predictions`. Write a sorting key that will enable you have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [158]:
# Write your code here
next_word_predictions = []
for w in frequency_trigrams.keys():
    if w[0] == tokens[-2] and w[1] == tokens[-1]:
        next_word_predictions.append((w[2], frequency_trigrams[w]))

next_word_predictions.sort(key=lambda x: x[1], reverse=True)
next_word_predictions = next_word_predictions[:cand_nbr]


In [159]:
next_word_predictions

[('stor', 34), ('liten', 26), ('gammal', 19), ('god', 17), ('sådan', 17)]

Finally, let us suppose that the user has typed _Det var en g_, rank the five possible candidates. Assign these predictions in a list that you will call `current_word_predictions_2`

In [160]:
current_text = "Det var en g".lower()

In [161]:
# Write your code here
tokens = tokenize(current_text)

current_word_predictions_2 = []
for w in frequency_trigrams.keys():
    if w[0] == tokens[-3] and w[1] == tokens[-2] and w[2].startswith(tokens[-1]):
        current_word_predictions_2.append((w[2], frequency_trigrams[w]))

current_word_predictions_2.sort(key=lambda x: (-x[1], x[0]))
current_word_predictions_2 = current_word_predictions_2[:cand_nbr]

In [162]:
current_word_predictions_2

[('gammal', 19), ('god', 17), ('gång', 10), ('ganska', 3), ('glädje', 2)]

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will:
1. Write a short individual report on your program. You will include the __regular expression you used to segment the text__ and the __unigram and bigram tables__ for _Det var en gång en katt som hette Nils_ and __two other sentences__.
2. Execute the Jupyter notebook by Peter Norvig here: <a href="http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb">https://nbviewer.jupyter.org/url/norvig.com/ipython/How to Do Things with Words.ipynb</a>. Just run all the cells and be sure that you understand the code. You will find the data here: <a href="http://norvig.com/ngrams/">http://norvig.com/ngrams/</a>.
3. In your report, after the description of your program, you will describe one experiment with Norvig's notebook and a __long string of words your will create yourself or copy from a text you like__. You will remove all the punctuation and white spaces from this string. You will set this string in lowercase letters. You will just add a cell at the end of Sect. 7 in Norvig's notebook, where you will use your string and run the notebook cell with the <tt>segment()</tt> and <tt>segment2()</tt> functions. __You will comment the segmentation results you obtained__ with the unigram and bigram models.

Submit your report as well as your notebook (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is September 22, 2023. You will have only three submission attempts. The deadline for the second and third ones are one week after you are noticed of your result.